# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv("/Users/victoriasandoval/python-api-challenge/WeatherPy/cities.csv")
city_df


,Unnamed: 0,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind,Country,Date
0,0,husavik,66.0449,-17.3389,41.95,90,100,2.51,IS,1652035899
1,1,uribia,11.7139,-72.2660,90.66,51,53,14.43,CO,1652036171
2,2,sitka,57.0531,-135.3300,49.98,66,25,5.55,US,1652036172
3,3,tiksi,71.6872,128.8694,-0.26,100,88,4.34,RU,1652036151
4,4,bredasdorp,-34.5322,20.0403,55.24,87,98,1.95,ZA,1652036143
...,...,...,...,...,...,...,...,...,...,...
547,547,hovd,48.0056,91.6419,36.79,46,9,13.44,MN,1652036438
548,548,pizhanka,57.4604,48.5420,48.38,66,56,9.62,RU,1652036439
549,549,arman,59.7000,150.1667,32.95,82,83,5.12,RU,1652036439
550,550,kolobrzeg,54.1757,15.5834,49.15,84,6,11.43,PL,1652036440


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = city_df[["Lat", "Lng"]].astype(float)
weights = city_df["Humidity"].astype(float)

In [5]:
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

In [6]:
fig = gmaps.figure()
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
ideal_cities = city_df.loc[(city_df["Temp"] > 70) & 
            (city_df["Temp"] < 80) &
            (city_df["Wind"] < 10) & 
            (city_df["Cloudiness"] == 0)].dropna()

ideal_cities


,Unnamed: 0,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind,Country,Date
52,52,abu kamal,34.4506,40.9171,71.64,30,0,6.02,SY,1652036192
53,53,rio grande,-32.0350,-52.0986,78.91,72,0,9.40,BR,1652035991
247,247,netarhat,23.4833,84.2667,71.64,34,0,1.16,IN,1652036283
288,288,angoche,-16.2325,39.9086,74.19,80,0,5.61,MZ,1652036302
408,408,hamilton,39.1834,-84.5333,70.75,50,0,5.01,US,1652036223


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = ideal_cities.copy()
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind,Country,Date
52,52,abu kamal,34.4506,40.9171,71.64,30,0,6.02,SY,1652036192
53,53,rio grande,-32.0350,-52.0986,78.91,72,0,9.40,BR,1652035991
247,247,netarhat,23.4833,84.2667,71.64,34,0,1.16,IN,1652036283
288,288,angoche,-16.2325,39.9086,74.19,80,0,5.61,MZ,1652036302
408,408,hamilton,39.1834,-84.5333,70.75,50,0,5.01,US,1652036223


In [9]:
hotel_df["Hotel Name"] = ""
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind,Country,Date,Hotel Name
52,52,abu kamal,34.4506,40.9171,71.64,30,0,6.02,SY,1652036192,
53,53,rio grande,-32.0350,-52.0986,78.91,72,0,9.40,BR,1652035991,
247,247,netarhat,23.4833,84.2667,71.64,34,0,1.16,IN,1652036283,
288,288,angoche,-16.2325,39.9086,74.19,80,0,5.61,MZ,1652036302,
408,408,hamilton,39.1834,-84.5333,70.75,50,0,5.01,US,1652036223,


In [10]:
params = {"radius": 5000, "types": "lodging", "key": g_key}

In [11]:
for index, row in hotel_df.iterrows():
    Lat = row["Lat"]
    Long = row["Lng"]

    params["location"] = f"{Lat},{Long}"

    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(url, params = params).json()

    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        print("not found, skip")

In [12]:
hotel_df

,Unnamed: 0,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind,Country,Date,Hotel Name
52,52,abu kamal,34.4506,40.9171,71.64,30,0,6.02,SY,1652036192,البوكمال
53,53,rio grande,-32.0350,-52.0986,78.91,72,0,9.40,BR,1652035991,Hotel Atlântico Rio Grande
247,247,netarhat,23.4833,84.2667,71.64,34,0,1.16,IN,1652036283,HOTEL PRAKASH
288,288,angoche,-16.2325,39.9086,74.19,80,0,5.61,MZ,1652036302,Guest House HESADA
408,408,hamilton,39.1834,-84.5333,70.75,50,0,5.01,US,1652036223,Six Acres Bed & Breakfast


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
marker_locations = hotel_df[["Lat", "Lng"]]

# Display figure
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))